# My introduction into Algorithmic Trading: A Not-So-Dumb Strategy

### Gut Trading

Now, when I started trading, I first tried to trade using my gut. This was probably the biggest mistake I've ever made, but it also taught me a lot about trading and how small I was in comparison with big institutions and educated, experienced people that were making billions. Once my "gambling" phase made me hit the rock bottom of my portfolio, I realized some kind of strategy needed to be made. My gut, it turned out, was mostly concerned with what to have for lunch.

### The First "Brilliant" Idea: The 99% Win-Rate to Ruin

The first idea I had for a strategy was a simple one: trade small and safe. Sounds smart, right? It seemed simple at first, until you have a 99% win rate with a P/L of -500. The issue I recognized was that my wins were too small to cover my one, catastrophic loss. I was picking up pennies in front of a steamroller.

Since I wanted to optimize this strategy with programming, I needed a way to quantify my decisions. This is where the real journey began. For this exploration, I will mostly be using `backtesting.py`, `pandas`, `matplotlib`, and `numpy`.

### Step 1: Asking the Right Question - What's a "Normal" Price Move?

My first real question was simple: how much does the price *actually* move up or down within a single candle, statistically speaking? If I could figure that out, maybe I could set a Take Profit and Stop Loss that wasn't just a random guess.

First, let's get the boring setup out of the way. We need to install and import some libraries to grab our data.

In [ ]:
%pip install yfinance pandas matplotlib

In [ ]:
# GetDiffs.note
import yfinance as yf
import pandas as pd

# Let's look at Gold Futures (GC=F). Why gold? Because it's cool.
# We'll grab one month of hourly data.
tickers = ["GC=F"]
period = "1mo"
interval = "1h"

data = yf.download(tickers=tickers, period=period, interval=interval)

# Let's get our standard candle data
o = data["Open"]
h = data["High"]
l = data["Low"]
c = data["Close"]

# How far did the price shoot up from the open?
h_diff = h - o

# And how far did it plummet from the open?
l_diff = o - l

print("\nHere are the raw price movements up and down from the open:")
h_diff, l_diff # Return differences

### Step 2: Let's Visualize This Chaos

Now that we have these raw numbers, they don't mean much on their own. We need to see them as a distribution to understand which price movements happen the most often. The idea is to set our take-profit (TP) and stop-loss (SL) based on what's probable.

Let's plot this chaos.

In [ ]:
# PlotDiffs.note
import matplotlib.pyplot as plt

# We'll create two plots (subplots), one for the ups and one for the downs.
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
bins = 75 # How many bars our histogram will have. More bins = more detail.

# --- First subplot: The Green Zone (Upward Movement) ---
ax1.hist(h_diff, bins=bins, color='green')
ax1.set_title("Distribution of Upward Price Moves (Open to High)")
ax1.set_xlabel("Price Difference ($)")
ax1.set_ylabel("Frequency")

# --- Second subplot: The Red Zone (Downward Movement) ---
ax2.hist(l_diff, bins=bins, color='red')
ax2.set_title("Distribution of Downward Price Moves (Open to Low)")
ax2.set_xlabel("Price Difference ($)")
ax2.set_ylabel("Frequency")

plt.tight_layout() # Makes sure our titles and labels don't overlap
plt.show()

### Step 3: The Normalization Epiphany - From Dollars to Percentages

Immediately, I hit a snag. As you can see from the charts, the most common move is a few dollars. That's great for Gold, but what if I wanted to trade Bitcoin, where a move is hundreds of dollars? Or a stock that moves by cents? This strategy isn't universal at all. The raw dollar value is useless.

For this, I needed a universal yardstick. The solution was simple normalization: looking at these movements as a *percentage* of the opening price.

In [ ]:
# Let's turn those raw dollar moves into percentages
# (How much it moved) / (What it started at)
h_pct = h_diff / o
l_pct = l_diff / o

print("Here are the percentage movements:")
h_pct, l_pct

And if we plot this new, improved data, it should give us a much more universal picture.

In [ ]:
# PlotPcts.note
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
bins = 75

# --- First subplot: The Green Zone (Percentage) ---
ax1.hist(h_pct, bins=bins, color='green')
ax1.set_title("Normalized Distribution of Upward Price Moves (in %)")
ax1.set_xlabel("Percentage Move (%)")
ax1.set_ylabel("Frequency")

# --- Second subplot: The Red Zone (Percentage) ---
ax2.hist(l_pct, bins=bins, color='red')
ax2.set_title("Normalized Distribution of Downward Price Moves (in %)")
ax2.set_xlabel("Percentage Move (%)")
ax2.set_ylabel("Frequency")

plt.tight_layout()
plt.show()

### Step 4: Quantifying "Probably" with Quantiles

This is much better. Now I have a general idea of where to put my TP/SLs on any asset. I could put my TP where a move is most likely, and my SL where it's less so. But "eyeballing it" isn't exactly a robust strategy. I needed to quantify how often these values *actually* appear.

That's why we bring in the professionals: **quantiles**.

In simple terms, a quantile tells you the value below which a certain percentage of your data falls. For example, the 0.5 quantile (the median) is the value that is bigger than 50% of all other values. This gives us a concrete number to work with.

In [ ]:
# Let's find the 50th percentile (the median)
percentage = 0.5

h_quantile = h_pct.quantile(percentage)
l_quantile = l_pct.quantile(percentage)

print(f"The 50% quantile for upward moves is: {h_quantile:.4f}%")
print(f"The 50% quantile for downward moves is: {l_quantile:.4f}%")

h_quantile, l_quantile

### Conclusion: A Foundation is Laid... But is the House Stable?

So, what does this all mean? In theory, the numbers above tell us that 50% of the time, the price moves up by `h_quantile` or less, and down by `l_quantile` or less. This gives us a statistical foundation. We can now say, "I'll set my Take Profit at the 50% quantile because it's a commonly reached level."

This is a huge leap from trading on a gut feeling. We've used data to find a probable range of movement for an asset.

Of course, this is just the beginning. The idea that this resembles a "chance" of being hit is a massive oversimplification (as we'll find out). The next logical step is to take this concept and build a simple strategy around it to backtest. Only then will we see if this statistical observation can actually be turned into a profitable (or at least, not disastrous) trading bot. The foundation is laid, but now we have to actually build the house and see if it stands up.